In [31]:
from scipy.stats import multivariate_normal
import numpy as np

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib auto



import sys
sys.path.insert(0, '..')

import ImageTools as it

Using matplotlib backend: Qt5Agg


In [ ]:
def gauss_c(x, m=0, s=1):
    return (1/np.sqrt(2*np.pi*s**2)) * np.exp(-(x-m)**2/(2*s**2))
    
def gauss_d(x, m=0, s=1):
    None

def I(n, t):
    
    np.factorial(m)
    
def gamma(n):
    return np.factorial(n-1)
    
    

In [ ]:
x = np.arange(11)-5

plt.plot(x, gauss_c(x))
plt.show()

In [ ]:
from astropy.io import fits
import os
import sep

from scipy.stats import multivariate_normal

In [34]:
def make_nonnegative(img):
    epsilon = np.abs(img.min()) + 1e-3
    img[img<=0] = img[img>0].min()
    return img

def get_random_spheroids(num=10):
    with open('../spheroids', 'r') as f:
        spheroids = np.array(f.readlines())
    
    if num==-1:
        num = len(spheroids)
    
    selected = np.random.choice(spheroids, num, replace=False)
    
    data_dir = os.path.join(os.getenv('HOME'), 'Documents/astro_data/orig_images')
    fmask = 'GDS_{}_{}.fits'
    f_string = os.path.join(data_dir, fmask)

    sources = []
    for s in selected:
        img = fits.getdata(f_string.format(s.strip(), 'h'))
        segmap = fits.getdata(f_string.format(s.strip(), 'segmap'))
        img_id = int(s.split('_')[1])
        sources.append((img.copy(), segmap.copy(), img_id))
        del img 
        del segmap
    
    return sources

def get_random_disks(num=10):
    with open('../disks', 'r') as f:
        spheroids = np.array(f.readlines())
    
    if num==-1:
        num = len(spheroids)
    
    selected = np.random.choice(spheroids, num, replace=False)
    
    data_dir = os.path.join(os.getenv('HOME'), 'Documents/astro_data/orig_images')
    fmask = 'GDS_{}_{}.fits'
    f_string = os.path.join(data_dir, fmask)

    sources = []
    for s in selected:
        img = fits.getdata(f_string.format(s.strip(), 'h'))
        segmap = fits.getdata(f_string.format(s.strip(), 'segmap'))
        img_id = int(s.split('_')[1])
        sources.append((img.copy(), segmap.copy(), img_id))
        del img 
        del segmap
    
    return sources


def denoise(img, segmap, img_id):
    noise_bank = img[segmap==0].flatten()
    other_source = np.logical_and(segmap!=0, segmap!=img_id)
    np.place(img, other_source, noise_bank)
    
    array_sep_likes = img.byteswap().newbyteorder()
    bkg = sep.Background(array_sep_likes, mask=segmap==img_id, bw=10, bh=10)
    img = img-bkg
    
    return img

In [43]:
source = get_random_disks(1)

In [44]:
img, segmap, img_id = source[0]
img = denoise(img, segmap, img_id)[21:63, 21:63]
#img[segmap!=img_id] = 0

#img += np.abs(img.min())
#img /= img.sum()

plt.imshow(img, cmap='gray')

ys, xs = np.zeros(img.shape[0]), np.zeros(img.shape[1])
for y in range(img.shape[0]):
    for x in range(img.shape[1]):
        ys[y] += img[y,x]
        xs[x] += img[y,x]

ys /= ys.sum()
xs /= xs.sum()

def gaus(x,a,x0,sigma):
    return a*exp(-(x-x0)**2/(2*sigma**2))

def fit_gauss(vals):
    xs = np.arange(vals.shape[0])
    mu = np.mean(vals*xs)
    sigma = np.mean(vals * (xs-mu)**2)

    popt,pcov = curve_fit(gaus,np.arange(vals.shape[0]),vals,p0=[1,mu,sigma])
    return popt
    
y_scale, y_mean, y_sigma = fit_gauss(ys)
y_fit = gaus(np.arange(img.shape[0]),y_scale, y_mean, y_sigma)

plt.figure()
plt.plot(ys)
plt.plot(y_fit)



x_scale, x_mean, x_sigma = fit_gauss(xs)
x_fit = gaus(np.arange(img.shape[0]),x_scale, x_mean, x_sigma)

plt.figure()
plt.plot(xs)
plt.plot(x_fit)

cov = np.cov(y_fit, x_fit)
mu = [y_mean, x_mean]

print(mu)
print(cov)

NameError: name 'curve_fit' is not defined

In [46]:
img, segmap, img_id = source[0]
print(img_id)
img = denoise(img, segmap, img_id)
src_map = segmap==img_id
#src_map = np.ones_like(img)

#img /= img.sum()
plt.figure(figsize=(10, 10))
plt.imshow(img, cmap='gray')

mu = it.img_center(img, src_map)
cov = it.img_cov(img, src_map)

print(mu)
print(cov)

y, x = np.arange(img.shape[0]), np.arange(img.shape[1])
X, Y = np.meshgrid(x, y)
pos = np.empty(X.shape + (2,))
pos[:, :, 0] = X
pos[:, :, 1] = Y
rv =  multivariate_normal(mu, cov)
plt.figure(figsize=(10, 10))

plt.contourf(X, Y, rv.pdf(pos))

11854
(42.247250277149313, 41.678711986845208)
[[ 13.41195575   5.73644961]
 [  5.73644961   9.46234879]]


In [35]:
spheroids = get_random_spheroids(-1)
disks = get_random_disks(-1)

In [42]:
c1, c2, c3 = [], [], []
for img, segmap, img_id in spheroids:
    img = denoise(img, segmap, img_id)
    cov = it.img_cov(img, segmap==img_id)
    c1.append(cov[0,0])
    c2.append(cov[1,1])
    c3.append(cov[0,1])
    
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(c1, c2, c3, color='b')

c1, c2, c3 = [], [], []
for img, segmap, img_id in disks:
    img = denoise(img, segmap, img_id)
    cov = it.img_cov(img, segmap==img_id)
    c1.append(cov[0,0])
    c2.append(cov[1,1])
    c3.append(cov[0,1])
    
ax.scatter(c1, c2, c3, color='r', alpha=0.2)


plt.show()
    

In [47]:
c1, c2 = [], []
for img, segmap, img_id in spheroids:
    img = denoise(img, segmap, img_id)
    cov = it.img_cov(img, segmap==img_id)
    c1.append(cov[0,0])
    c2.append(cov[1,1])
    
plt.scatter(c1, c2, color='b')
    
c1, c2 = [], []
for img, segmap, img_id in disks:
    img = denoise(img, segmap, img_id)
    cov = it.img_cov(img, segmap==img_id)
    c1.append(cov[0,0])
    c2.append(cov[1,1])
    
plt.scatter(c1, c2, color='r')


plt.show()
    

In [69]:
xs = []
ys = []


c1, c2, c3, c4 = [], [], [], []
for img, segmap, img_id in spheroids:
    img = denoise(img, segmap, img_id)
    src_map = segmap==img_id
    cov = it.img_cov(img, src_map)
    evals, evecs = np.linalg.eig(cov)
    pairs = {}
    for i in range(2):
        pairs[evals[i]] = evecs[:, i]
    
    eval_maj = evals.max()
    eval_min = evals.min()

    major_x, major_y = pairs[eval_maj]  # Eigenvector with largest eigenvalue
    minor_x, minor_y = pairs[eval_min]
    
    xs.append(np.array([cov[0,0], cov[1,1], np.sqrt(evals.min()/evals.max()), src_map.sum()]))
    ys.append(1)
    c1.append(cov[0,0])
    c2.append(cov[1,1])
    c3.append(np.sqrt(evals.min()/evals.max()))
    c4.append(src_map.sum())
    
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(c1, c2, c4, color='b', alpha=0.2)

c1, c2, c3, c4 = [], [], [], []
for img, segmap, img_id in disks:
    img = denoise(img, segmap, img_id)
    src_map = segmap==img_id
    cov = it.img_cov(img, src_map)
    evals, evecs = np.linalg.eig(cov)
    pairs = {}
    for i in range(2):
        pairs[evals[i]] = evecs[:, i]
    
    eval_maj = evals.max()
    eval_min = evals.min()

    major_x, major_y = pairs[eval_maj]  # Eigenvector with largest eigenvalue
    minor_x, minor_y = pairs[eval_min]
    
    xs.append(np.array([cov[0,0], cov[1,1], np.sqrt(evals.min()/evals.max()), src_map.sum()]))
    ys.append(0)
    c1.append(cov[0,0])
    c2.append(cov[1,1])
    c3.append(np.sqrt(evals.min()/evals.max()))
    c4.append(src_map.sum())
    
ax.scatter(c1, c2, c4, color='r', alpha=0.2)


plt.show()
    

In [84]:
from sklearn.linear_model import LogisticRegression

c = LogisticRegression()

c = c.fit(xs, ys)

corr = c.predict(xs)==ys
corr.sum()/len(corr)

0.92535392535392536

In [83]:
from sklearn.mixture import BayesianGaussianMixture

c = BayesianGaussianMixture(n_components=2, max_iter=1000)


c = c.fit(xs, ys)
print(c.predict(xs)[:5])

corr = c.predict(xs)==ys
corr.sum()/len(corr)

[0 1 0 1 0]


0.5714285714285714